Xarrays is elusive so this is where I am going to document the basics with my dataset(s)

In [2]:
import xarray as xr
import rioxarray

In [22]:
# Start with Geotiff data
geotiff_file = '/Users/hyin/yin_usgs/turkiye_2023/surfacedef2LOS/observation-files/usgs-isce-geotiffs/s1_2023018-20230209_p14_azimuth.tif'
da = rioxarray.open_rasterio(geotiff_file)          # Make a data array with rioxarray
da

<xarray.DataArray (band: 1, y: 12019, x: 9681)>
[116355939 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 35.28 35.28 35.28 35.28 ... 37.97 37.97 37.97 37.97
  * y            (y) float64 38.77 38.77 38.77 38.77 ... 35.43 35.43 35.43 35.43
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     3.4028235e+38
    scale_factor:   1.0
    add_offset:     0.0

In [21]:
# Covert our xarray.DataArray into a xarray.Dataset
ds = da.to_dataset('band')          # Take the band object and set it as a variable
ds = ds.rename({1: 'los','x': 'lon','y': 'lat'})          # Rename the variable to a more useful name
ds

<xarray.Dataset>
Dimensions:      (lat: 12019, lon: 9681)
Coordinates:
  * lon          (lon) float64 35.28 35.28 35.28 35.28 ... 37.97 37.97 37.97
  * lat          (lat) float64 38.77 38.77 38.77 38.77 ... 35.43 35.43 35.43
    spatial_ref  int64 0
Data variables:
    los          (lat, lon) float32 ...
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     3.4028235e+38
    scale_factor:   1.0
    add_offset:     0.0

## Joining Arrays

In [14]:
# JOIN ARRAYS EXAMPLE

import numpy as np
import pandas as pd
import xarray as xr

In [20]:
# how does linspace work again?
# array between 35 and 36, with 11 points, making it step by 0.1
np.linspace(35,36,11)

array([35. , 35.1, 35.2, 35.3, 35.4, 35.5, 35.6, 35.7, 35.8, 35.9, 36. ])

In [75]:
# Let's make up some synthetic data to make a stand-in synthetic dataset
los_synth = np.random.randint(1,255,size=(11,16)) # Create 11x16 numpy array
lon_synth = np.linspace(35,36,11)
lat_synth = np.linspace(35,36.5,16)

los_synth.shape # confirm that it's 11x16 numpy array

ds_synth = xr.Dataset(
       data_vars=dict(
           los_synth=(('x', 'y'), los_synth)),
       coords=dict(
           x=lon_synth,
           y=lat_synth)
)
ds_synth.los_synth.shape
ds_synth

<xarray.Dataset>
Dimensions:    (x: 11, y: 16)
Coordinates:
  * x          (x) float64 35.0 35.1 35.2 35.3 35.4 ... 35.6 35.7 35.8 35.9 36.0
  * y          (y) float64 35.0 35.1 35.2 35.3 35.4 ... 36.1 36.2 36.3 36.4 36.5
Data variables:
    los_synth  (x, y) int64 150 124 83 185 29 130 29 ... 171 80 12 38 168 1 94

In [71]:
# Let's make up some observed data to make a stand-in observed dataset
los_obs = np.random.randint(1,255,size=(35,42)) # Create 11x16 numpy array
lon_obs = np.linspace(35,36.2,35)
lat_obs = np.linspace(35,36.5,42)

los_obs.shape # confirm that it's 11x16 numpy array

ds_obs = xr.Dataset(
       data_vars=dict(
           los_obs=(('x', 'y'), los_obs)),
       coords=dict(
           x=lon_obs,
           y=lat_obs)
)
ds_obs.x

<xarray.DataArray 'x' (x: 35)>
array([35.      , 35.035294, 35.070588, 35.105882, 35.141176, 35.176471,
       35.211765, 35.247059, 35.282353, 35.317647, 35.352941, 35.388235,
       35.423529, 35.458824, 35.494118, 35.529412, 35.564706, 35.6     ,
       35.635294, 35.670588, 35.705882, 35.741176, 35.776471, 35.811765,
       35.847059, 35.882353, 35.917647, 35.952941, 35.988235, 36.023529,
       36.058824, 36.094118, 36.129412, 36.164706, 36.2     ])
Coordinates:
  * x        (x) float64 35.0 35.04 35.07 35.11 35.14 ... 36.09 36.13 36.16 36.2

In [73]:
# Now let's try interpolating over the NaNs
# ds_synth.los_synth.interp_like
interp_synth = ds_synth.los_synth.interp_like(ds_obs)
interp_synth

<xarray.DataArray 'los_synth' (x: 35, y: 42)>
array([[180.        , 150.        , 120.        , ...,  64.46341463,
         63.73170732,  63.        ],
       [121.05882353, 105.77905308,  90.49928264, ...,  89.10903874,
         95.99569584, 102.88235294],
       [ 62.11764706,  61.55810617,  60.99856528, ..., 113.75466284,
        128.25968436, 142.76470588],
       ...,
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan]])
Coordinates:
  * x        (x) float64 35.0 35.04 35.07 35.11 35.14 ... 36.09 36.13 36.16 36.2
  * y        (y) float64 35.0 35.04 35.07 35.11 35.15 ... 36.39 36.43 36.46 36.5

In [70]:
# Combine, keeping the coordinate points used by the observed data
# And filling in NaNs for the synthetic data
# the resulting synthetic los data array will have the same shape as the observed data array
# xr.combine_by_coords([x3, x1], join="override")
# ds_merge = xr.combine_by_coords([ds_obs, ds_synth], join='left')

ds_merge = xr.combine_by_coords([ds_synth,ds_obs], join='outer')

ds_merge

<xarray.Dataset>
Dimensions:    (x: 44, y: 56)
Coordinates:
  * x          (x) float64 35.0 35.04 35.07 35.1 ... 36.09 36.13 36.16 36.2
  * y          (y) float64 35.0 35.04 35.07 35.1 35.11 ... 36.4 36.43 36.46 36.5
Data variables:
    los_obs    (x, y) float64 199.0 10.0 221.0 nan 154.0 ... nan 7.0 76.0 146.0
    los_synth  (x, y) float64 180.0 nan nan 98.0 nan nan ... nan nan nan nan nan